In [ ]:
import pandas as pd
from cyvcf2 import VCF

In [ ]:
vcf = list(VCF('/Users/quinlan/Documents/Git/SEMIColon/data/test_donor091_snp_chunk_chr21.vcf'))


In [ ]:
from collections import defaultdict

# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

# Initialize the dictionary to hold the count of each configuration
config_counts = defaultdict(int)

# Loop through each variant in the VCF
for v in vcf:
    # Get the genotypes for the samples
    genotypes = v.gt_types

    # Assign sample context
    youngsperm = genotypes[0]
    blood = genotypes[1]
    oldsperm = genotypes[2]

    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10:
        if blood == youngsperm and blood == oldsperm:
            continue
        elif blood == UNKNOWN:
            continue
        elif youngsperm == UNKNOWN and oldsperm == UNKNOWN:
            continue
        elif blood == youngsperm and oldsperm == UNKNOWN:
            continue
        elif blood == oldsperm and youngsperm == UNKNOWN:
            continue
        else:
    # Create a tuple of the genotype configuration
            config = (youngsperm, blood, oldsperm)

    # Increment the count for this configuration
            config_counts[config] += 1


# Print the configuration counts
for config, count in config_counts.items():
    print(f"Configuration {config}: {count} occurrences")

In [ ]:
def get_info_v(v, blood_col, sperm1_col, sperm2_col):
    genotypes = v.gt_types
    alt_depths = v.gt_alt_depths
    ref_depths = v.gt_ref_depths
    bases = v.gt_bases

    #assign sample context: GT
    youngsperm = genotypes[sperm1_col]
    blood = genotypes[blood_col]
    oldsperm = genotypes[sperm2_col]

    #assign sample context: AD
    ys_altdepth = alt_depths[sperm1_col]
    os_altdepth = alt_depths[sperm2_col]
    blood_altdepth = alt_depths[blood_col]

    #assign sample context: ref_depths
    ys_refdepth = ref_depths[sperm1_col]
    os_refdepth = ref_depths[sperm2_col]
    blood_refdepth = ref_depths[blood_col]

    #assign sample context: ref_depths
    ys_bases = bases[sperm1_col]
    os_bases = bases[sperm2_col]
    blood_bases = bases[blood_col]

    return youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases

In [8]:
def get_interesting(v, youngsperm, blood, oldsperm):
    if blood == youngsperm and blood == oldsperm:
        return False
    elif blood == UNKNOWN:
        return False
    elif youngsperm == UNKNOWN and oldsperm == UNKNOWN:
        return False
    elif blood == youngsperm and oldsperm == UNKNOWN:
        return False
    elif blood == oldsperm and youngsperm == UNKNOWN:
        return False
    else:
        return True


In [10]:
# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

blood_col = 1
sperm1_col = 0
sperm2_col = 2

# variant.gt_types, variant.gt_ref_depths, variant.gt_alt_depths # numpy arrays
# variant.gt_phases, variant.gt_quals, variant.gt_bases # numpy array

for v in vcf:
    youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases = get_info_v(v,
                                                                                                                        blood_col, sperm1_col, sperm2_col)
    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10 and get_interesting(v, youngsperm, blood, oldsperm):
        print("blood", blood_bases, "alt depth", blood_altdepth, "ref depth", blood_refdepth, "young sperm", ys_bases, "alt depth", ys_altdepth,
                  "ref depth", ys_refdepth, "old sperm",os_bases, "alt depth", os_altdepth, "ref depth", os_refdepth)
        print(v.INFO.get("AC"))
            #print(v.gt_types, ys_altdepth, os_altdepth, v.QUAL)
            #print(os_refdepth, ys_refdepth, blood_refdepth)

blood T/C alt depth 3 ref depth 1 young sperm T/C alt depth 3 ref depth 4 old sperm T/T alt depth 0 ref depth 3
2
blood T/G alt depth 1 ref depth 2 young sperm T/G alt depth 4 ref depth 3 old sperm G/G alt depth 2 ref depth 0
4
blood G/A alt depth 1 ref depth 2 young sperm G/A alt depth 4 ref depth 4 old sperm A/A alt depth 2 ref depth 0
4
blood A/T alt depth 2 ref depth 2 young sperm A/T alt depth 4 ref depth 5 old sperm A/A alt depth 0 ref depth 1
2
blood C/G alt depth 4 ref depth 5 young sperm C/G alt depth 5 ref depth 4 old sperm C/C alt depth 0 ref depth 1
2
blood C/A alt depth 3 ref depth 4 young sperm C/A alt depth 4 ref depth 5 old sperm A/A alt depth 2 ref depth 0
4
blood T/T alt depth 1 ref depth 2 young sperm T/A alt depth 6 ref depth 3 old sperm T/T alt depth 0 ref depth 2
1
blood G/G alt depth 1 ref depth 2 young sperm G/A alt depth 6 ref depth 3 old sperm A/A alt depth 1 ref depth 0
3
blood C/C alt depth 1 ref depth 1 young sperm C/G alt depth 4 ref depth 5 old sperm C/C 